In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define dataset path (update with your actual path)
dataset_path = "/content/drive/MyDrive/Stenosis detection/arcade"
os.listdir(dataset_path)  # List dataset files


Mounted at /content/drive


['stenosis',
 'syntax',
 'weights',
 'weights_CBAM+optimization+noaug',
 'dataset2.yaml',
 'train_masks',
 'val_masks',
 'train_images',
 'weights_yolov9+CBAM+SimCLR',
 'test_masks',
 'val_images',
 'efficientnet_stenosis.keras',
 'resnet_stenosis.keras',
 'train',
 'val',
 'test',
 'dataset3.yaml',
 'dataset.yaml',
 'model.pt',
 'mobilenet_stenosis.keras',
 'vgg_stenosis.keras',
 'inceptionV3_stenosis.keras']

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
from ultralytics import YOLO

# Load trained YOLOv9 model with best weights
yolo_model = YOLO("/content/drive/MyDrive/Stenosis detection/arcade/weights_yolov9+CBAM+SimCLR/best.pt")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import json
import numpy as np

def load_ground_truth(json_path):
    """
    Loads ground truth bounding boxes from the dataset JSON file.
    """
    with open(json_path, 'r') as f:
        data = json.load(f)

    ground_truth = {}

    for image in data['images']:
        img_id = image['id']
        img_filename = image['file_name']

        # Get bounding boxes for this image
        bboxes = []
        for ann in data['annotations']:
            if ann['image_id'] == img_id:
                bbox = ann['bbox']  # COCO format: [xmin, ymin, width, height]
                xmin, ymin, w, h = bbox
                xmax, ymax = xmin + w, ymin + h
                bboxes.append([xmin, ymin, xmax, ymax, 1])  # Class "stenosis" = 1

        ground_truth[img_filename] = np.array(bboxes)  # Store as numpy array

    return ground_truth


In [5]:
json_file=f"{dataset_path}/stenosis/test/annotations/test.json"
gt= load_ground_truth(json_file)

In [6]:
def yolo_detect(image_path):
   results = yolo_model(image_path)  # Run inference
   # Extract bounding boxes from YOLO's output
   detected_boxes = []
   for result in results:
    for box in result.boxes.data:
      xmin, ymin, xmax, ymax, confidence, class_id = box.cpu().numpy()
      detected_boxes.append([xmin, ymin, xmax, ymax, confidence, int(class_id)])
   return {image_path: detected_boxes} # List of detected boxes

In [8]:
from tensorflow.keras.models import load_model

# Load Trained EfficientNet Model
mobilenet_model = load_model("/content/drive/MyDrive/Stenosis detection/arcade/vgg_stenosis.keras")


In [9]:
import cv2
def predict_stenosis(image_path, threshold=0.2):
    """
    Runs EfficientNet Stenosis Classification on a Full Angiogram Image.
    """
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (512, 512)) / 255.0  # Normalize
    image_input = np.expand_dims(image_resized, axis=0)  # Add batch dimension

    prediction = mobilenet_model.predict(image_input)[0][0]  # Get probability

    if prediction > 0.5:
        return 1  # Stenosis Detected
    else:
        return 0  # No Stenosis


In [10]:
def detect_stenosis(image_path):

    # Step 1: Classify the Full Image Using EfficientNet
    stenosis_detected = predict_stenosis(image_path)

    # Step 2: If No Stenosis, Return No Detections
    if stenosis_detected == 0:
        return []

    # Step 3: Otherwise, Run YOLOv9 for Object Detection
    results = yolo_model(image_path)
    final_detections = []

    for result in results:
        for box in result.boxes.data:
            xmin, ymin, xmax, ymax, confidence, class_id = box.cpu().numpy()

            # Keep only Stenosis (Class 0) Detections
            if class_id == 0:
                final_detections.append([xmin, ymin, xmax, ymax, confidence])

    return final_detections


In [11]:
image_path = "/content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/1.png"



In [ ]:
import cv2
from google.colab.patches import cv2_imshow
detections= detect_stenosis(image_path)
def visualize_detections(image_path, detections):

    image = cv2.imread(image_path)

    for (xmin, ymin, xmax, ymax, confidence) in detections:
        cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        cv2.putText(image, f"Stenosis ({confidence:.2f})", (int(xmin), int(ymin) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2_imshow( image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [12]:
import os

def run_combined_model_on_dataset(test_folder, gt_boxes):

    combined_detections = {}

    for filename in os.listdir(test_folder):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(test_folder, filename)
            detections = detect_stenosis(image_path)

            combined_detections[filename] = detections

    return combined_detections


In [13]:

test_folder = "/content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/"
final_detections = run_combined_model_on_dataset(test_folder, gt)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/10.png: 512x512 1 stenosis, 907.1ms
Speed: 11.3ms preprocess, 907.1ms inference, 30.5ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/139.png: 512x512 1 stenosis, 502.8ms
Speed: 2.8ms preprocess, 502.8ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/118.png: 512x512 1 stenosis, 450.7ms
Speed: 2.2ms preprocess, 450.7ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/108.png: 512x512 1 stenosis, 710.2ms
Speed: 2.4ms prep

In [14]:
import os

# Path to the test images folder
test_folder = "/content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/"

# Create a dictionary where each image has a label 1 (Stenosis Present)
gt_labels = {filename: 1 for filename in os.listdir(test_folder) if filename.endswith((".png", ".jpg"))}



In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score


predictions = []
true_labels = []

for filename, label in gt_labels.items():
    image_path = test_folder + filename
    pred = 1 if detect_stenosis(image_path) else 0  # If YOLO detects anything, classify as 1

    predictions.append(pred)
    true_labels.append(label)



1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/10.png: 512x512 1 stenosis, 432.9ms
Speed: 2.3ms preprocess, 432.9ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/139.png: 512x512 1 stenosis, 471.5ms
Speed: 2.0ms preprocess, 471.5ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/118.png: 512x512 1 stenosis, 439.6ms
Speed: 2.6ms preprocess, 439.6ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

image 1/1 /content/drive/MyDrive/Stenosis detection/arcade/stenosis/test/images/108.png: 512x512 1 stenosis, 439.1ms
Speed: 2.0ms prepro

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Compute Metrics
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
accuracy = accuracy_score(true_labels, predictions)


print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, Accuracy: {accuracy:.4f}")

Precision: 1.0000, Recall: 0.5233, F1-score: 0.6871, Accuracy: 0.5233


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(true_labels, predictions))



Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.52      0.69       300

    accuracy                           0.52       300
   macro avg       0.50      0.26      0.34       300
weighted avg       1.00      0.52      0.69       300



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
